# View Training Metrics

**Author: Shahzad Sanjrani**

**Date: 21.11.24**

This is a way to quickly produce training metrics for a specific `version_x` for experimental/test purposes. 

In [22]:
import warnings
# ignore ugly ass warning --> doesn't work...
with warnings.catch_warnings(action="ignore"):
    from tensorflow.python.summary.summary_iterator import summary_iterator
    
import os
import pandas as pd
import mplhep as hep
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm

In [23]:
def convert_tb_data(root_dir, sort_by=None):
    """Convert local TensorBoard data into Pandas DataFrame.
    
    Function takes the root directory path and recursively parses
    all events data.    
    If the `sort_by` value is provided then it will use that column
    to sort values; typically `wall_time` or `step`.
    
    *Note* that the whole data is converted into a DataFrame.
    Depending on the data size this might take a while. If it takes
    too long then narrow it to some sub-directories.
    
    Paramters:
        root_dir: (str) path to root dir with tensorboard data.
        sort_by: (optional str) column name to sort by.
    
    Returns:
        pandas.DataFrame with [wall_time, name, step, value] columns.

    Note: originally from https://laszukdawid.com/blog/2021/01/26/parsing-tensorboard-data-locally/
    """

    def convert_tfevent(filepath):
        return pd.DataFrame([
            parse_tfevent(e) for e in summary_iterator(filepath) if len(e.summary.value)
        ])

    def parse_tfevent(tfevent):
        return dict(
            wall_time=tfevent.wall_time,
            name=tfevent.summary.value[0].tag,
            step=tfevent.step,
            value=float(tfevent.summary.value[0].simple_value),
        )
    
    columns_order = ['wall_time', 'name', 'step', 'value']
    
    out = []
    # originally os.walk but i don't see the point...
    for filename in glob.glob(f"{root_dir}/*"):
        if "events.out.tfevents" not in filename:
            continue
        file_full_path = os.path.join(filename)
        out.append(convert_tfevent(file_full_path))

    # Concatenate (and sort) all partial individual dataframes
    all_df = pd.concat(out)[columns_order]
    if sort_by is not None:
        all_df = all_df.sort_values(sort_by)
        
    return all_df.reset_index(drop=True)

In [3]:
def plot_hepfig(figsize=(10,10)):
    ''' plt.figure but use the mplhep package to make it look nice '''
    hep.style.use("CMS")
    mpl.rcParams["font.size"] = 20
    plt.figure(figsize=figsize)

In [17]:
def plot_output(df, metric, extra=None, save = None, verbose=False):
    '''
    Given metric in df, plot metric
    '''
    metric_arr = df[ df['name'] == metric ]['value']
    x_arr = df[ df['name'] == metric ]['step']

    plot_hepfig()
    plt.plot(x_arr, metric_arr, label=metric if extra is None else f"{extra}_{metric}")
    plt.legend()
    plt.grid()

    if save is not None:
        os.makedirs(save, exist_ok=True)
        metric_name = metric.replace("/", "-") # otherwise tedious
        fpath = os.path.join(save, f"{metric_name}.pdf")
        plt.savefig(fpath)
        plt.close()
        if verbose: print(f"saved {fpath}")

## User input here

In [30]:
store_dir = f"/nfs/dust/cms/user/sanjrani/SPANet_Investigations/investigation2/pepper_analysis/output/h4t_systematics/spanet/models/spanet_output"
model = f"version_5"

df = convert_tb_data(os.path.join(store_dir, model))
# print(df['name'].unique())

In [12]:
# some ideas:
# 'loss/zpt1/assignment_loss' 'loss/zpt2/assignment_loss'
# 'loss/nzpt1/assignment_loss' 'loss/nzpt2/assignment_loss'
# 'loss/zpt1/detection_loss' 'loss/zpt2/detection_loss'
# 'loss/nzpt1/detection_loss' 'loss/nzpt2/detection_loss' 'loss/total_loss'
# 'Purity/*tzp*nztp/event_purity' 'validation_accuracy'
# 'Purity/*tzp*nztp/event_proportion' 'Purity/*tzp*nztp/tzp_purity'
# 'Purity/*tzp*nztp/nztp_purity' 'Purity/*tzp0nztp/event_purity'
# 'Purity/*tzp0nztp/event_proportion' 'Purity/*tzp0nztp/tzp_purity'
# 'Purity/*tzp1nztp/event_purity' 'Purity/*tzp1nztp/event_proportion'
# 'Purity/*tzp1nztp/tzp_purity' 'Purity/*tzp1nztp/nztp_purity'
# 'Purity/*tzp2nztp/event_purity' 'Purity/*tzp2nztp/event_proportion'
# 'Purity/*tzp2nztp/tzp_purity' 'Purity/*tzp2nztp/nztp_purity'
# 'Purity/0tzp*nztp/event_purity' 'Purity/0tzp*nztp/event_proportion'
# 'Purity/0tzp*nztp/nztp_purity' 'Purity/0tzp0nztp/event_purity'
# 'Purity/0tzp0nztp/event_proportion' 'Purity/0tzp1nztp/event_purity'
# 'Purity/0tzp1nztp/event_proportion' 'Purity/0tzp1nztp/nztp_purity'
# 'Purity/0tzp2nztp/event_purity' 'Purity/0tzp2nztp/event_proportion'
# 'Purity/0tzp2nztp/nztp_purity' 'Purity/1tzp*nztp/event_purity'
# 'Purity/1tzp*nztp/event_proportion' 'Purity/1tzp*nztp/tzp_purity'
# 'Purity/1tzp*nztp/nztp_purity' 'Purity/1tzp0nztp/event_purity'
# 'Purity/1tzp0nztp/event_proportion' 'Purity/1tzp0nztp/tzp_purity'
# 'Purity/1tzp1nztp/event_purity' 'Purity/1tzp1nztp/event_proportion'
# 'Purity/1tzp1nztp/tzp_purity' 'Purity/1tzp1nztp/nztp_purity'
# 'Purity/1tzp2nztp/event_purity' 'Purity/1tzp2nztp/event_proportion'
# 'Purity/1tzp2nztp/tzp_purity' 'Purity/1tzp2nztp/nztp_purity'
# 'Purity/2tzp*nztp/event_purity' 'Purity/2tzp*nztp/event_proportion'
# 'Purity/2tzp*nztp/tzp_purity' 'Purity/2tzp*nztp/nztp_purity'
# 'Purity/2tzp0nztp/event_purity' 'Purity/2tzp0nztp/event_proportion'
# 'Purity/2tzp0nztp/tzp_purity' 'Purity/2tzp1nztp/event_purity'
# 'Purity/2tzp1nztp/event_proportion' 'Purity/2tzp1nztp/tzp_purity'
# 'Purity/2tzp1nztp/nztp_purity' 'Purity/2tzp2nztp/event_purity'
# 'Purity/2tzp2nztp/event_proportion' 'Purity/2tzp2nztp/tzp_purity'
# 'Purity/2tzp2nztp/nztp_purity' 'jet/accuracy_1_of_1'
# 'jet/accuracy_1_of_2' 'jet/accuracy_2_of_2' 'jet/accuracy_1_of_3'
# 'jet/accuracy_2_of_3' 'jet/accuracy_3_of_3' 'jet/accuracy_1_of_4'
# 'jet/accuracy_2_of_4' 'jet/accuracy_3_of_4' 'jet/accuracy_4_of_4'
# 'particle/accuracy_1_of_1' 'particle/accuracy_1_of_2'
# 'particle/accuracy_2_of_2' 'particle/accuracy_1_of_3'
# 'particle/accuracy_2_of_3' 'particle/accuracy_3_of_3'
# 'particle/accuracy_1_of_4' 'particle/accuracy_2_of_4'
# 'particle/accuracy_3_of_4' 'particle/accuracy_4_of_4' 'particle/accuracy'
# 'particle/sensitivity' 'particle/specificity' 'particle/f_score'

In [17]:
# some ideas:
#  'loss/t1/assignment_loss' 'loss/t2/assignment_loss'
#  'loss/t3/assignment_loss' 'loss/t4/assignment_loss'
#  'loss/t1/detection_loss' 'loss/t2/detection_loss'
#  'loss/t3/detection_loss' 'loss/t4/detection_loss' 'loss/total_loss'
#  'epoch' 'Purity/*t/event_purity' 'Purity/*t/event_proportion'
#  'Purity/*t/t_purity' 'Purity/0t/event_purity'
#  'Purity/0t/event_proportion' 'Purity/1t/event_purity'
#  'Purity/1t/event_proportion' 'Purity/1t/t_purity'
#  'Purity/2t/event_purity' 'Purity/2t/event_proportion'
#  'Purity/2t/t_purity' 'Purity/3t/event_purity'
#  'Purity/3t/event_proportion' 'Purity/3t/t_purity'
#  'Purity/4t/event_purity' 'Purity/4t/event_proportion'
#  'Purity/4t/t_purity' 'jet/accuracy_1_of_1' 'jet/accuracy_1_of_2'
#  'jet/accuracy_2_of_2' 'jet/accuracy_1_of_3' 'jet/accuracy_2_of_3'
#  'jet/accuracy_3_of_3' 'jet/accuracy_1_of_4' 'jet/accuracy_2_of_4'
#  'jet/accuracy_3_of_4' 'jet/accuracy_4_of_4' 'particle/accuracy_1_of_1'
#  'particle/accuracy_1_of_2' 'particle/accuracy_2_of_2'
#  'particle/accuracy_1_of_3' 'particle/accuracy_2_of_3'
#  'particle/accuracy_3_of_3' 'particle/accuracy_1_of_4'
#  'particle/accuracy_2_of_4' 'particle/accuracy_3_of_4'
#  'particle/accuracy_4_of_4' 'particle/accuracy' 'particle/sensitivity'
#  'particle/specificity' 'particle/f_score' 'validation_accuracy'

In [28]:
save_dir = os.path.join(store_dir, model, "training_metrics")
plot_output(df, 'jet/accuracy_2_of_4', save = save_dir)

## Quick and dirty

Plot all the relevant metrics

In [31]:
save_dir = os.path.join(store_dir, model, "training_metrics")

metrics_filter = ['jet', 'Purity', 'loss', 'particle', 'accuracy']
training_metrics = []

for name in tqdm(df['name'].unique()):

    for mfilter in metrics_filter:
        if mfilter in name:
            training_metrics.append(name)
            plot_output(df, name, save = save_dir, extra=model, verbose=False)
            break

# print(training_metrics)

100%|██████████| 534/534 [00:08<00:00, 64.70it/s]  
